Scripts del Proyect (UFC WINNER PREDICTION)

Script 1: Preparacion de datos para el entrenamiento

In [1]:
#Import Cell
#used to import all the libraries and functions used
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import sys, warnings, os

In [2]:
#To ignore max-iteration warnings while cross validating scores
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
#Setting columns and rows to display all the results
pd.set_option("display.max_columns", None, "display.max_rows", None)

In [ ]:
# Leemos los archivos csv
def read_file_csv(filename):
    df = pd.read_csv(os.path.join('../data/raw/', filename)).set_index('Winner')
    print(filename, ' cargado correctamente')
    return df

In [ ]:
# Realizamos la transformación de datos
def data_preparation(ufc_master_ds):
    ufc_master_ds['draw_diff'] = (ufc_master_ds['BlueDraws']-ufc_master_ds['RedDraws'])
    ufc_master_ds['avg_sig_str_pct_diff'] = (ufc_master_ds['BlueAvgSigStrPct']-ufc_master_ds['RedAvgSigStrPct'])
    ufc_master_ds['avg_TD_pct_diff'] = (ufc_master_ds['BlueAvgTDPct']-ufc_master_ds['RedAvgTDPct'])
    ufc_master_ds['win_by_Decision_Majority_diff'] = (ufc_master_ds['BlueWinsByDecisionMajority']-ufc_master_ds['RedWinsByDecisionMajority'])
    ufc_master_ds['win_by_Decision_Split_diff'] = (ufc_master_ds['BlueWinsByDecisionSplit']-ufc_master_ds['RedWinsByDecisionSplit'])
    ufc_master_ds['win_by_Decision_Unanimous_diff'] = (ufc_master_ds['BlueWinsByDecisionUnanimous']-ufc_master_ds['RedWinsByDecisionUnanimous'])
    ufc_master_ds['win_by_TKO_Doctor_Stoppage_diff'] = (ufc_master_ds['BlueWinsByTKODoctorStoppage']-ufc_master_ds['RedWinsByTKODoctorStoppage'])
    ufc_master_ds['odds_diff'] = (ufc_master_ds['BlueOdds']-ufc_master_ds['RedOdds'])
    ufc_master_ds['ev_diff'] = (ufc_master_ds['BlueExpectedValue']-ufc_master_ds['RedExpectedValue'])

In [ ]:
# After extracting the necessary information from these variables, there's no more need for them. So, I'll just drop them
#Dropping variables
var_drop = [
'BlueOdds',
'RedOdds',
'BlueCurrentLoseStreak', 'RedCurrentLoseStreak',
'BlueCurrentWinStreak', 'RedCurrentWinStreak',
'BlueLongestWinStreak', 'RedLongestWinStreak',
'BlueWins', 'RedWins',
'BlueLosses', 'RedLosses',
'BlueTotalRoundsFought', 'RedTotalRoundsFought',
'BlueTotalTitleBouts', 'RedTotalTitleBouts',
'BlueWinsByKO', 'RedWinsByKO',
'BlueWinsBySubmission', 'RedWinsBySubmission',
'BlueHeightCms', 'RedHeightCms',
'BlueReachCms', 'RedReachCms',
'BlueAge', 'RedAge',
'BlueAvgSigStrLanded', 'RedAvgSigStrLanded',
'BlueAvgSubAtt', 'RedAvgSubAtt',
'BlueAvgTDLanded', 'RedAvgTDLanded',
'BlueDraws','BlueAvgSigStrPct','BlueAvgTDPct','BlueWinsByDecisionMajority','BlueWinsByDecisionSplit','BlueWinsByDecisionUnanimous','BlueWinsByTKODoctorStoppage',
'RedDraws','RedAvgSigStrPct','RedAvgTDPct','RedWinsByDecisionMajority','RedWinsByDecisionSplit','RedWinsByDecisionUnanimous','RedWinsByTKODoctorStoppage']
ufc_master_ds.drop(var_drop, axis=1, inplace = True)

In [ ]:
comm_drop = [
'Date','Location','Country','WeightClass','Gender','NumberOfRounds','EmptyArena','Finish','FinishDetails','FinishRound','FinishRoundTime','TotalFightTimeSecs','BlueWeightLbs','RedWeightLbs'
]
ufc_master_ds.drop(comm_drop, axis=1, inplace = True)

In [ ]:
  #It has one spelling mistake
    ufc_master_ds['BlueStance'].loc[ufc_master_ds['BlueStance']=='Switch '] = 'Switch'

    stance = ['BlueStance', 'RedStance']

    for x in stance:
            ufc_master_ds[x] = [4 if st == 'Orthodox'
                            else 3 if st == 'Southpaw'
                            else 2 if st == 'Switch'
                            else 1 for st in ufc_master_ds[x]]

In [ ]:
ufc_master_ds['BlueStance'].loc[ufc_master_ds['BlueStance']=='Switch '] = 'Switch'

In [ ]:
stance = ['BlueStance', 'RedStance']

In [ ]:
for x in stance:
    ufc_master_ds[x] = [4 if st == 'Orthodox'
                           else 3 if st == 'Southpaw'
                           else 2 if st == 'Switch'
                           else 1 for st in ufc_master_ds[x]]
#using -1 and 1 for both red and blue so there is no misunderstanding that one variable is better than the other    
ufc_master_ds['BetterRank'] = [-1 if rank == 'Red'
                               else 1 if rank == 'Blue'
                               else 0 for rank in ufc_master_ds['BetterRank']]

ufc_master_ds['TitleBout'] = [1 if tb==True else 0 for tb in ufc_master_ds['TitleBout']]

In [ ]:
ufc_master_ds['Stance_diff'] = (ufc_master_ds['BlueStance'] - ufc_master_ds['RedStance'])
ufc_master_ds.drop(stance, axis = 1, inplace = True)

In [ ]:
#Encoding label so it is easier to find correlation
ufc_master_ds['Winner'] = [1 if winner == 'Red' else 0 for winner in ufc_master_ds.Winner]

In [ ]:
##dataset final
ufc_master_ds.drop(ufc_master_ds.loc[:,'BMatchWCRank':'BetterRank'], axis=1, inplace = True)
ufc_master_ds.to_csv("../data/processed/ufc_master_train.csv")

Script 2: Código de Entrenamiento


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import sys, warnings, os
import pickle

In [ ]:
# Cargar la tabla transformada
df = pd.read_csv("../data/processed/ufc_master_train.csv").set_index('Winner')

In [ ]:
label = df.Winner
df.drop(['Winner'], axis=1, inplace = True)

In [ ]:
#Encoding the remaining categorical variables
cat_col = ['RedFighter', 'BlueFighter']
enc = LabelEncoder()
for i in df[cat_col]:
    df[i] = enc.fit_transform(df[i])
    print('Transformación de datos completa')
    return df, label

In [ ]:
# Exportamos la matriz de datos con las columnas seleccionadas
def data_exporting(df, features, filename):
    dfp = df[features]
    dfp.to_csv(os.path.join('../data/processed/', filename))
    print(filename, 'exportado correctamente en la carpeta processed')